# Import libraries

In [145]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from Utils import *
from data_augmentation import *
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import os

In [146]:
# verify if GPU is available
gpu_available = tf.test.is_gpu_available()
is_cuda_gpu_available = tf.test.is_gpu_available(cuda_only=True)
is_cuda_gpu_min_3 = tf.test.is_gpu_available(True, (3,0))
print("gpu_available: ", gpu_available, "\tis_cuda_gpu_available: ", is_cuda_gpu_available)

gpu_available:  True 	is_cuda_gpu_available:  True


2023-12-19 23:24:01.153196: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-19 23:24:01.153212: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-12-19 23:24:01.153275: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-19 23:24:01.153279: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-12-19 23:24:01.153308: I tensor

# Import data

In [147]:
# Load the data
training_data = np.load('training_dataset/training_data.npy')
valid_periods = np.load('training_dataset/valid_periods.npy')
categories = np.load('training_dataset/categories.npy')

# Preprocess data

## Build sequences

In [149]:
dataset, labels = build_sequences_optimized(training_data, valid_periods)

padded sequences:  0
Dataset shape:  (21000, 200)


## Split dataset

In [150]:
train_data, train_labels, test_data, test_labels = split_dataset(dataset, labels)

Train data shape:  (16800, 200)
Test data shape:  (4200, 200)


# Models

## LSTM

In [152]:
# Build LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(200, 1)))
model.add(Dense(18))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(train_data, train_labels, epochs=50, batch_size=512)

Epoch 1/50
33/33 [==============================] - 2s 42ms/step - loss: 0.1590
Epoch 2/50
33/33 [==============================] - 1s 38ms/step - loss: 0.0229
Epoch 3/50
33/33 [==============================] - 1s 38ms/step - loss: 0.0190
Epoch 4/50
33/33 [==============================] - 1s 37ms/step - loss: 0.0182
Epoch 5/50
33/33 [==============================] - 1s 37ms/step - loss: 0.0177
Epoch 6/50
33/33 [==============================] - 1s 38ms/step - loss: 0.0173
Epoch 7/50
33/33 [==============================] - 1s 38ms/step - loss: 0.0169
Epoch 8/50
33/33 [==============================] - 1s 37ms/step - loss: 0.0167
Epoch 9/50
33/33 [==============================] - 1s 37ms/step - loss: 0.0165
Epoch 10/50
33/33 [==============================] - 1s 37ms/step - loss: 0.0164
Epoch 11/50
33/33 [==============================] - 1s 37ms/step - loss: 0.0163
Epoch 12/50
33/33 [==============================] - 1s 38ms/step - loss: 0.0163
Epoch 13/50
33/33 [==================

# Evaluate model

In [157]:
telescope_musk = 9
# Make predictions on the test data
model.summary()
predictions = model.predict(test_data)
final_predictions = predictions[:, :telescope_musk]
final_test_labels = test_labels[:, :telescope_musk]

result = evaluate_model(final_test_labels, final_predictions)

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 50)                10400     
                                                                 
 dense_9 (Dense)             (None, 18)                918       
                                                                 
Total params: 11318 (44.21 KB)
Trainable params: 11318 (44.21 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
132/132 [==============================] - 1s 10ms/step
MAE: 0.06917916097850943
MSE: 0.011076279320111043
RMSE: 0.10524390395700381
MAPE: 264883457.27433792%


In [ ]:
for n in range(10):
    plot_predictions(test_data, predictions, test_labels, n+100)

In [159]:
model.save("models/lstm_model_VX.h5")